# **Pipeline for object detection and tracking in 3D:**

### **Required packages:**

In [9]:
#!pip install -r requirements.txt 
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
from itertools import zip_longest


from pathlib import Path
from models.common import DetectMultiBackend
from utils.general import (Profile, cv2, non_max_suppression, scale_boxes, xyxy2xywh)
from utils.augmentations import letterbox
from utils.torch_utils import select_device, smart_inference_mode

### **1. Definition of Objects and Functions**

##### 2D Object detection : YOLOV5

In [8]:
# DEFINE PARAMETERS:

weights="weights/X-512.pt"
source_1="data/view1"
source_2="data/view2"
device="cpu"
project="runs"
name="exp"

image_size=(512, 512)
conf_thres=0.25
max_det=20
line_thickness=2
iou_thres=0.45

save_txt=True
save_csv=False
nosave=False
hide_labels=False 
hide_conf=True

In [18]:
class ObjectDetector():
    
    def __init__(self, device, weights, source_1, source_2, image_size):
        self.device = select_device(device)
        self.model = DetectMultiBackend(weights, self.device)
        self.stride, self.names, self.pt = self.model.stride, self.model.names, self.model.pt
        self.imgsz = image_size
        self.model.warmup(imgsz=(1 , 3, *self.imgsz))
        self.dt = Profile(device=self.device)
        self.source_1 = Path(source_1)
        self.source_2 = Path(source_2)
        self.files_1 = [f for f in self.source_1.glob('*') if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
        self.files_2 = [f for f in self.source_2.glob('*') if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
        
    def __iter__(self):
        for file_1, file_2 in zip_longest(self.files_1, self.files_2, fillvalue=None):
            img_1 = cv2.imread(str(file_1))
            img_2 = cv2.imread(str(file_2))
            yield (img_1, img_2)
    
    @smart_inference_mode()
    def detect_object_2D(self, im):
        with self.dt:
            image_original = im
            im = letterbox(im, self.imgsz, stride=self.stride, auto=True)[0]
            im = im.transpose((2, 0, 1))[::-1]
            im = np.ascontiguousarray(im)
            im = torch.from_numpy(im).to(self.device)
            im = im.float().unsqueeze(0)
            im /= 255
                        
            pred = self.model(im)
            pred = non_max_suppression(pred)
            
            for det in pred:
                gn = torch.tensor(image_original.shape)[[1, 0, 1, 0]] #Normalization
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], image_original.shape).round() # Rescale to original size
                for *xyxy, conf, cls in reversed(det):
                    coords = ((xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist())  # normalized xywh
                    line = (cls, *coords)
                    print(line)


In [19]:
detector = ObjectDetector(device, weights, source_1, source_2, image_size)

YOLOv5  v7.0-383-g1435a8ee Python-3.10.5 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 484 layers, 88417530 parameters, 0 gradients


##### Z ESTIMATION:

In [ ]:
def calculate_distance_to_object(left_bbox, right_bbox, im_left, im_right, focal_length = 707.0493, baseline = 0.06):
    """
    Calculate the distance to an object using bounding box coordinates from left and right images,
    along with the camera calibration parameters (focal length and baseline).
    
    Parameters:
        left_bbox (tuple): Bounding box coordinates in the left image (left, top, right, bottom)
        right_bbox (tuple): Bounding box coordinates in the right image (left, top, right, bottom)
        focal_length (float): Focal length of the camera (in pixels)
        baseline (float): Baseline distance between cameras (in meters)
        img_left (ndarray): Left image (grayscale)
        img_right (ndarray): Right image (grayscale)
        
    Returns:
        float: Distance to the object (in meters)
        float: Disparity between the left and right images (in pixels)
    """
    
    # Get the center of the bounding box in the left and right images
    x_left = (left_bbox[0] + left_bbox[2]) / 2  # Average x of the left box
    y_left = (left_bbox[1] + left_bbox[3]) / 2  # Average y of the left box
    
    x_right = (right_bbox[0] + right_bbox[2]) / 2  # Average x of the right box
    y_right = (right_bbox[1] + right_bbox[3]) / 2  # Average y of the right box
    
    # Calculate disparity (horizontal pixel difference between the left and right image)
    disparity = abs(x_left - x_right)
    
    if disparity == 0:
        return float('inf'), disparity  # If disparity is zero, the object is too far away or at the same location
    
    # Calculate the distance to the object using the formula
    distance = (focal_length * baseline) / disparity
    
    return distance, disparity

In [21]:
for i, images in enumerate(detector):
    print(f'Image {i}')
    detector.detect_object_2D(images[0])
    print('\n')
    detector.detect_object_2D(images[1])
    print('\n')

Image 0
(tensor(3.), 0.10171568393707275, 0.6202702522277832, 0.2034313678741455, 0.7594594359397888)


(tensor(1.), 0.4473039209842682, 0.4486486613750458, 0.025326797738671303, 0.13513512909412384)
(tensor(1.), 0.38153594732284546, 0.47837838530540466, 0.02124183066189289, 0.18378378450870514)
(tensor(1.), 0.16421568393707275, 0.43378376960754395, 0.013071895577013493, 0.062162160873413086)
(tensor(1.), 0.42197713255882263, 0.44594594836235046, 0.01715686358511448, 0.1459459513425827)
(tensor(1.), 0.4554738700389862, 0.4472973048686981, 0.018790850415825844, 0.13783784210681915)
(tensor(1.), 0.31086599826812744, 0.7324324250221252, 0.09068627655506134, 0.47567567229270935)
(tensor(3.), 0.07598039507865906, 0.6243243217468262, 0.15196079015731812, 0.7513513565063477)


Image 1
(tensor(3.), 0.1021241843700409, 0.6202702522277832, 0.2042483687400818, 0.7594594359397888)


(tensor(1.), 0.42687907814979553, 0.44999998807907104, 0.023692810907959938, 0.1432432383298874)
(tensor(1.), 0.4428

In [ ]:
while True:
    # 1. New frame (2)
    frame = get_next_frame()
    
    # 2. Yolo detection in 2 stereo images
    detections_2d = yolo_model.detect(frame)
    
    # 3.3D reconstruction (center x,y in 2 stereo images)
    detections_3d = []
    for bbox in detections_2d:

        Z = disparity_to_depth(disparity_map[bbox_center_y1, bbox_center_x1, bbox_center_y2, bbox_center_x2 ])

        X = (bbox_center_x - cx) * Z / fx
        Y = (bbox_center_y - cy) * Z / fy
        detections_3d.append(point_3d)
    
    # 4 detect oclusion (if oclusion kalman, if not linear interpolation of line tracker.)
    
        # 4A. UPDATE tracker
        active_tracks = KALMAN.tracker.update(detections_3d)
        
        # 5A. visualize results:
        for track_id, track in active_tracks.items():
            position = track['kalman'].state[:3]  # Posición filtrada
            velocity = track['kalman'].state[3:]  # Velocidad estimada
            draw_track(frame, position, track_id)
            
        #4A linear interpolation